## MNIST exercise with hypertuning
**Goal: Introduction to Keras Tuner object**

**Exercise:**
    
1. Review the steps of the code in this notebook
2. Look for the build_model and build_model_hp functions 
3. run the notebook, review the summary of hypertuning results
4. change the build_model functions and add one or two more parameters 


In [5]:
!pip install keras_tuner --prefix ~/.local

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 1.5 MB/s eta 0:00:0000:01
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)

In [6]:

# ----------- IMPORT STATEMENTS ---------------
import numpy as np
np.random.seed(1)  # for reproducibility
 
from tensorflow import keras
import tensorflow as tf
import datetime, os

import keras_tuner as kt    #<<----- NEW import
#---------------------------------------------
print('import done')

import done


In [7]:
#Load MNIST data from Keras datasets
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()

X_train=X_train[0:1000,]  #only need smaller subset to get good results
Y_train=Y_train[0:1000,]

# --------- Reshape input data, b/c Keras expects N-3D images (ie 4D matrix)
X_train = X_train[:,:,:,np.newaxis]
X_test  = X_test[:,:,:,np.newaxis]

#Scale 0 to 1  - or should we not scale
X_train = X_train/255.0
X_test  = X_test/255.0

# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = keras.utils.to_categorical(Y_train, 10)
Y_test  = keras.utils.to_categorical(Y_test,  10)

# ------------- End loading and preparing data --------------
print('X train shape:', X_train.shape) 
print('X test shape:', X_test.shape) 


X train shape: (1000, 28, 28, 1)
X test shape: (10000, 28, 28, 1)


## Notice here is a new function for the hypertuner that wraps around the build-model function

'hp' is an argument is this function

hp is a hyperparameter tuner object

See https://keras.io/keras_tuner/

After running through the notebook, add a new parameter 

For example, add a set of choices for either 'relu' or 'elu' activation

For details see here
https://keras.io/api/keras_tuner/hyperparameters/#choice-method
and here
https://keras.io/api/layers/activations/

In [8]:
def build_model_hp(hp): 
  hp_numfilters    = hp.Int('hpnumfilters',min_value=8,max_value=32,step=4)
  #your variable name         ^^^ the parameter name in the hp object

  # For Step4  -------- Add code here (and see below) --------------------
  #  add a new parameter object; for example
  hp_Activation    = hp.Choice('hpActivation', values=['relu','elu']) 
    
  return build_model(hp_numfilters,hp_Activation)   #<<---- dont forget to pass the new choices to build_model


In [9]:
# --------------Set up Model ---------------------
def build_model(numfilters,activation_choice):   #<<------add code: if you add parameters to search, add them to the argument 
                                      #            list and change code to use those arguments
    mymodel = keras.models.Sequential()
    mymodel.add(keras.layers.Convolution2D(numfilters, 
                                       (3, 3),
                                       strides=1,  
                                       data_format="channels_last",
                                       activation=activation_choice, #<<<< ---- add code
                                       input_shape=(28,28,1))) 
    mymodel.add(keras.layers.Convolution2D(numfilters, 
                                       (3, 3),
                                       strides=1,  
                                       data_format="channels_last",
                                       activation=activation_choice))
    mymodel.add(keras.layers.MaxPooling2D(pool_size=(2,2),strides=2,data_format="channels_last"))
    mymodel.add(keras.layers.Flatten())            #reorganize 2DxFilters output into 1D
  
    #----------------Now add final classification layers
    mymodel.add(keras.layers.Dense(32, activation=activation_choice))   
    mymodel.add(keras.layers.Dense(10, activation='softmax'))
    print('assemble model done')
    
    # --------- Now configure model algorithm -----
    mymodel.compile(loss='categorical_crossentropy',
               optimizer=keras.optimizers.Adam(learning_rate=0.005),  
               metrics=['accuracy'])

    return mymodel

## Set up hypertuner object so that it will build model and run fit
https://keras.io/keras_tuner/




In [10]:
myES_function = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5) #patience before stopping


dirname        = 'My_HP_trials'   
num_max_epochs = 60    #max to train 1 model, set this something higher than expected'

tuner = kt.Hyperband(build_model_hp,
                     objective  = 'val_accuracy',
                     max_epochs = num_max_epochs,  
                     factor     = 3,
                     hyperband_iterations=10,
                     directory   =dirname, 
                     overwrite   =True,   #overwrite directory each run
                     project_name='hyperbandtest',
                     executions_per_trial=5,  #to try several initializations
                     seed        =777)

#this has same arguments as the model.fit function
tunerhistory=tuner.search(X_train, Y_train, 
          validation_data=(X_test,Y_test),
          batch_size=32, epochs=num_max_epochs, verbose=1,callbacks=[myES_function])
        
tuner.results_summary(5)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=5)
    

Trial 14 Complete [00h 00m 20s]
val_accuracy: 0.8914599895477295

Best val_accuracy So Far: 0.9150399923324585
Total elapsed time: 00h 03m 58s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in My_HP_trials/hyperbandtest
Showing 5 best trials
Objective(name="val_accuracy", direction="max")

Trial 0006 summary
Hyperparameters:
hpnumfilters: 28
hpActivation: relu
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 3
tuner/round: 0
Score: 0.9150399923324585

Trial 0007 summary
Hyperparameters:
hpnumfilters: 32
hpActivation: relu
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 3
tuner/round: 0
Score: 0.9109999895095825

Trial 0011 summary
Hyperparameters:
hpnumfilters: 16
hpActivation: relu
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 3
tuner/round: 0
Score: 0.9091600060462952

Trial 0001 summary
Hyperparameters:
hpnumfilters: 20
hpActivation: relu
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 3
tuner/round: 0
Score: 0.9067600131034851

Trial 0012 sum

In [11]:
#print("best to worst\n")
print("Info, best parameters: rank hpnumfilters activation")
for i in range(len(best_hps)):
        print("               ",i,best_hps[i].get('hpnumfilters'),  
                                best_hps[i].get('hpActivation') ) # <<<<----- add code here
              


Info, best parameters: rank hpnumfilters activation
                0 28 relu
                1 32 relu
                2 16 relu
                3 20 relu
                4 24 relu


In [13]:
 for trial in tuner.oracle.get_best_trials(): #[0].trial_id
        print('best trial:',trial.trial_id)
        


best trial: 0006


In [14]:
for i in range(tuner.oracle._get_num_brackets()):
   print('bracket',i,'num rounds',tuner.oracle._get_num_rounds(i))


bracket 0 num rounds 1
bracket 1 num rounds 2
bracket 2 num rounds 3
bracket 3 num rounds 4
